In [0]:
# ╔════════════════════════════════════════════════════════════════════════════════╗
# ║                           Bronze Layer Data Load                               ║
# ║                                                                                ║
# ║ Purpose:                                                                       ║
# ║   - Loads raw CRM & ERP source data into the Bronze layer for further ETL.     ║
# ║   - Ensures foundational data availability for downstream analytics.           ║
# ║                                                                                ║
# ║ Job Schedule:                                                                  ║
# ║   - This notebook is scheduled to run daily at 2:00 AM UTC.                    ║
# ║   - Automated via Databricks Jobs for reliability and consistency.             ║
# ║                                                                                ║
# ║ Notes:                                                                         ║
# ║   - All DDL/DML statements are executed from an external SQL script.           ║
# ║   - Execution logs and row counts are printed for transparency.                ║
# ╚════════════════════════════════════════════════════════════════════════════════╝

In [0]:
from datetime import datetime
import re

# ASCII Art for Bronze Layer Data Load Start
print(r"""
▀█████████▄     ▄████████  ▄██████▄  ███▄▄▄▄    ▄███████▄     ▄████████       ▄█          ▄████████ ▄██   ▄      ▄████████    ▄████████ 
  ███    ███   ███    ███ ███    ███ ███▀▀▀██▄ ██▀     ▄██   ███    ███      ███         ███    ███ ███   ██▄   ███    ███   ███    ███ 
  ███    ███   ███    ███ ███    ███ ███   ███       ▄███▀   ███    █▀       ███         ███    ███ ███▄▄▄███   ███    █▀    ███    ███ 
 ▄███▄▄▄██▀   ▄███▄▄▄▄██▀ ███    ███ ███   ███  ▀█▀▄███▀▄▄  ▄███▄▄▄          ███         ███    ███ ▀▀▀▀▀▀███  ▄███▄▄▄      ▄███▄▄▄▄██▀ 
▀▀███▀▀▀██▄  ▀▀███▀▀▀▀▀   ███    ███ ███   ███   ▄███▀   ▀ ▀▀███▀▀▀          ███       ▀███████████ ▄██   ███ ▀▀███▀▀▀     ▀▀███▀▀▀▀▀   
  ███    ██▄ ▀███████████ ███    ███ ███   ███ ▄███▀         ███    █▄       ███         ███    ███ ███   ███   ███    █▄  ▀███████████ 
  ███    ███   ███    ███ ███    ███ ███   ███ ███▄     ▄█   ███    ███      ███▌    ▄   ███    ███ ███   ███   ███    ███   ███    ███ 
▄█████████▀    ███    ███  ▀██████▀   ▀█   █▀   ▀████████▀   ██████████      █████▄▄██   ███    █▀   ▀█████▀    ██████████   ███    ███ 
               ███    ███                                                    ▀                                               ███    ███ 
""")
print("Starting Bronze Layer Data Load...\n" + "="*80)

with open(
  "/Workspace/Users/barathiselvanofficial@gmail.com/databrics-sql-datawarehouse-project/scripts/bronze/data_load_bronze.sql"
) as f:
    sql_text = f.read()

# Remove single-line comments
lines = [
    line for line in sql_text.split('\n')
    if not line.strip().startswith('--')
]

sql_text_no_comments = '\n'.join(lines)

statements = [
    stmt.strip()
    for stmt in sql_text_no_comments.split(';')
    if stmt.strip()
]

def get_sql_action_table(sql):
    sql = sql.strip()
    action = None
    table = None
    # Match TRUNCATE TABLE
    m = re.match(r"(?i)truncate\s+table\s+([^\s;]+)", sql)
    if m:
        action = "TRUNCATE"
        table = m.group(1)
    # Match INSERT INTO
    m = re.match(r"(?i)insert\s+into\s+([^\s(]+)", sql)
    if m:
        action = "INSERT"
        table = m.group(1)
    # Match CREATE TABLE
    m = re.match(r"(?i)create\s+table\s+([^\s(]+)", sql)
    if m:
        action = "CREATE"
        table = m.group(1)
    # Match DROP TABLE
    m = re.match(r"(?i)drop\s+table\s+([^\s;]+)", sql)
    if m:
        action = "DROP"
        table = m.group(1)
    # Match UPDATE
    m = re.match(r"(?i)update\s+([^\s]+)", sql)
    if m:
        action = "UPDATE"
        table = m.group(1)
    # Match DELETE FROM
    m = re.match(r"(?i)delete\s+from\s+([^\s]+)", sql)
    if m:
        action = "DELETE"
        table = m.group(1)
    return action, table

overall_start_time = datetime.now()

for idx, stmt in enumerate(statements, 1):
    print(f"\n{'='*40}\n🟢 Executing SQL Statement #{idx}:\n{'='*40}")
    print("Executing SQL statement...")
    start_time = spark.sql("SELECT current_timestamp() AS Start_Time").collect()[0]['Start_Time']
    print(f"⏰ Start Time: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
    try:
        action, table = get_sql_action_table(stmt)
        affected_rows = None
        if action == "TRUNCATE":
            # Get row count before truncate
            pre_count = spark.sql(f"SELECT COUNT(*) AS cnt FROM {table}").collect()[0]['cnt']
            spark.sql(stmt)
            affected_rows = pre_count
            print(f"Number of truncated rows: {affected_rows}")
        elif action == "INSERT":
            # Get row count before insert
            pre_count = spark.sql(f"SELECT COUNT(*) AS cnt FROM {table}").collect()[0]['cnt']
            spark.sql(stmt)
            # Get row count after insert
            post_count = spark.sql(f"SELECT COUNT(*) AS cnt FROM {table}").collect()[0]['cnt']
            affected_rows = post_count - pre_count
            print(f"Number of rows added: {affected_rows}")
        else:
            result = spark.sql(stmt)
            print(f"Number of rows affected: {result.count()}")
        end_time = spark.sql("SELECT current_timestamp() AS End_Time").collect()[0]['End_Time']
        print(f"⏰ End Time:   {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
        elapsed_time = (end_time - start_time).total_seconds()
        print(f"⏳ Elapsed Time: {elapsed_time:.2f} seconds")
        if action and table:
            print(f"🔹 {action} operation executed for table '{table}'.")
        else:
            print(f"\n🔹 Executing SQL statement #{idx}.")
        print("✅ Statement Execution Successful\n" + '='*80)
    except Exception as e:
        print(f"\n{'='*40}\n❌ Error Executing Statement #{idx}\n{'='*40}")
        print(f"Error: {e}")
        print(f"Type: {type(e).__name__}")
        print(f"Args: {e.args}")
        print(f"Details: {getattr(e, 'detail', 'No additional details')}")
        print(f"Traceback: {getattr(e, '__traceback__', 'No traceback available')}\n{'='*80}")

overall_end_time = datetime.now()
total_elapsed = (overall_end_time - overall_start_time).total_seconds()
print(f"\n{'#'*80}")
print("🎉 DDL Statement run for the Bronze Layer is completed.")
print(r"""
▀█████████▄     ▄████████  ▄██████▄  ███▄▄▄▄    ▄███████▄     ▄████████       ▄█          ▄████████ ▄██   ▄      ▄████████    ▄████████ 
  ███    ███   ███    ███ ███    ███ ███▀▀▀██▄ ██▀     ▄██   ███    ███      ███         ███    ███ ███   ██▄   ███    ███   ███    ███ 
  ███    ███   ███    ███ ███    ███ ███   ███       ▄███▀   ███    █▀       ███         ███    ███ ███▄▄▄███   ███    █▀    ███    ███ 
 ▄███▄▄▄██▀   ▄███▄▄▄▄██▀ ███    ███ ███   ███  ▀█▀▄███▀▄▄  ▄███▄▄▄          ███         ███    ███ ▀▀▀▀▀▀███  ▄███▄▄▄      ▄███▄▄▄▄██▀ 
▀▀███▀▀▀██▄  ▀▀███▀▀▀▀▀   ███    ███ ███   ███   ▄███▀   ▀ ▀▀███▀▀▀          ███       ▀███████████ ▄██   ███ ▀▀███▀▀▀     ▀▀███▀▀▀▀▀   
  ███    ██▄ ▀███████████ ███    ███ ███   ███ ▄███▀         ███    █▄       ███         ███    ███ ███   ███   ███    █▄  ▀███████████ 
  ███    ███   ███    ███ ███    ███ ███   ███ ███▄     ▄█   ███    ███      ███▌    ▄   ███    ███ ███   ███   ███    ███   ███    ███ 
▄█████████▀    ███    ███  ▀██████▀   ▀█   █▀   ▀████████▀   ██████████      █████▄▄██   ███    █▀   ▀█████▀    ██████████   ███    ███ 
               ███    ███                                                    ▀                                               ███    ███ 
""")
print(f"⏳ Total Execution Time: {total_elapsed:.2f} seconds")
print(f"{'#'*80}\n")